In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import librosa
import numpy as np

from uss.inference import separate

/home/alexh/miniconda3/envs/uss/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_normalized_energy(filepath):
    data, sample_rate = librosa.load(filepath)


    print(type(data))

    energy = sum(abs(data**2))


    duration = librosa.get_duration(y=data, sr=sample_rate)

    print(f"Duration: {duration}")
    print(len(data) / sample_rate)

    normalized_energy = energy / duration
    print("Normalized energy of the audio file:", normalized_energy)

In [4]:
AUDIO_NO_BACKGROUND_MUSIC = "/inworld/tts_dataset/podcast_host_audio_combined_filtered/alec_baldwin/wavs/amy_schumer_grew_up_in_a_nude_house/256.wav"

In [5]:
MUSIC_AUDIO_FILE = "/home/alexh/uss/separated_results/0/level=1/Music.wav"

SILNECE_AUDIO_FILE = ""

In [6]:
get_normalized_energy(MUSIC_AUDIO_FILE)

<class 'numpy.ndarray'>
Duration: 13.09002267573696
13.09002267573696
Normalized energy of the audio file: 141.82085281589883


In [8]:
class Args:
    def __init__(self, audio_path, output_dir):
        self.audio_path = audio_path
        self.condition_type = "at_soft"
        self.levels = [1]  # 1 is most coarse level
        self.class_ids = []
        self.queries_dir = ""
        self.query_emb_path = ""
        self.output_dir = output_dir

        # config, checkpoint TODO
        self.config_yaml = "./scripts/train/ss_model=resunet30,querynet=at_soft,data=full.yaml"
        self.checkpoint_path = "pretrained.ckpt"

In [11]:
args = Args(AUDIO_NO_BACKGROUND_MUSIC, "uss_output")

separate(args)

Using cpu.
Sample rate: 44100
Loading model ...
Separating ...
------ alexLevel 1 ------
Human sounds 380.87124790070504
Animal 0.0
Music 2.032117472275493
Natural sounds 0.0
Sounds of things 0.0
Source-ambiguous sounds 0.0
Channel, environment and background 0.0


In [ ]:
import json

In [13]:
with open("wav_file_and_label_to_energy_level_adam_carolla.json", "r") as f:
    wav_file_and_label_to_energy_level = json.load(f)

print(len(wav_file_and_label_to_energy_level))

67781


In [26]:
from pprint import pprint


In [29]:
def print_and_get_ratio(filepath):
    item = [x for x in wav_file_and_label_to_energy_level if x[0] == filepath][0]
    print(item[0])
    pprint(item[1])

    print(f"Human to music ratio: {item[1]['Human sounds'] / item[1]['Music']}")

In [28]:
print_and_get_ratio('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_1_reenactments_of_jeffrey_toobin_and_orland_brown_incidents/313.wav')

{'Animal': 0.0,
 'Channel, environment and background': 0.0,
 'Human sounds': 4135.169450043683,
 'Music': 3021.0386530763662,
 'Natural sounds': 0.0,
 'Sounds of things': 0.0,
 'Source-ambiguous sounds': 0.0}
Human to music ratio: 1.368790646168258


In [15]:
[x for x in wav_file_and_label_to_energy_level if x[0] == "/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/baltimore_schools_news_and_rotten_tomatoes_game__live_in/263.wav"]

[['/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/baltimore_schools_news_and_rotten_tomatoes_game__live_in/263.wav',
  {'Human sounds': 207.05128609385244,
   'Animal': 0.0,
   'Music': 64.58546706960317,
   'Natural sounds': 0.0,
   'Sounds of things': 0.0,
   'Source-ambiguous sounds': 0.0,
   'Channel, environment and background': 0.0}]]

In [4]:
music_files = [(x[0], x[1]['Music']) for x in wav_file_and_label_to_energy_level if x[1]['Music'] != 0]

print(len(music_files))

5219


In [5]:
music_files.sort(key=lambda x: x[1], reverse=True)

In [6]:
music_files[:10]

[('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_1_reenactments_of_jeffrey_toobin_and_orland_brown_incidents/313.wav',
  3021.0386530763662),
 ('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/richard_belzer__deep_sea_neptune_societypart_1_carolla_classics/608.wav',
  2018.5562217625952),
 ('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/richard_belzer__deep_sea_neptune_societypart_1_carolla_classics/292.wav',
  1871.8712980919443),
 ('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/john_mellencamp_toolbox__mark_geragos_part_1_carolla_classics/448.wav',
  1653.043576917877),
 ('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/carolla_classics_megans_law_competition__2014_ace_awards/189.wav',
  1389.372234507189),
 ('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_2_jonathan_kite__schmoes_know_carolla_classics/391.wav',
  1300.4807893541677),
 ('/inworld/tts_data

In [45]:
for m in music_files[5000:5010]:
    print_and_get_ratio(m[0])

/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_1_masked_singer_reveal__ray_stevens_acs_november/665.wav
{'Animal': 383.28991193379125,
 'Channel, environment and background': 0.0,
 'Human sounds': 913.224255586847,
 'Music': 0.005094562077933048,
 'Natural sounds': 0.0,
 'Sounds of things': 0.0,
 'Source-ambiguous sounds': 0.0}
Human to music ratio: 179254.71151729647
/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/super_bowl__halftime_show_recap_acs_february_14/78.wav
{'Animal': 0.0,
 'Channel, environment and background': 0.0,
 'Human sounds': 918.7029315522719,
 'Music': 0.005073450854961856,
 'Natural sounds': 0.0,
 'Sounds of things': 0.0,
 'Source-ambiguous sounds': 0.0}
Human to music ratio: 181080.48305105325
/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/jesse_ventura__the_buried_life_cast_part_2_carolla/380.wav
{'Animal': 0.0,
 'Channel, environment and background': 0.0,
 'Human sounds': 1694.5772259931155,
 'Mus

In [12]:
for x in music_files[-4000:-3990]:
    print(x)


('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_1_jay_lenos_accident__joel_stein_acs_november/269.wav', 65.13445218999314)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_1_2022_ace_awards_acs_december_15/587.wav', 65.04136267634773)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/tim_allen__ernie_hudson_part_2_carolla_classics/214.wav', 64.71795486168533)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_2_diana_maria_riva__news_acs_december_1/351.wav', 64.71791996267046)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/part_1_the_light_we_carry__tales_from_the/532.wav', 64.6386437543656)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/tia_carrere__michael_yo_acs_february_7/356.wav', 64.61952335035087)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/baltimore_schools_news_and_rotten_tomatoes_game__live_in/263.wav', 64.5854670696

In [32]:
human_to_music_ratio_non_zero_music = [(x[0], x[1]['Human sounds'] / x[1]['Music'], x[1]['Human sounds'], x[1]['Music']) for x in wav_file_and_label_to_energy_level if x[1]['Music'] != 0]


In [33]:
human_to_music_ratio_non_zero_music[0]

('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/tucker_out_at_fox__don_lemon_out_at_cnn/5.wav',
 6.340389231440749,
 835.9910337589206,
 131.8516897375014)

In [34]:
human_to_music_ratio_non_zero_music.sort(key=lambda x: x[1], reverse=False)

In [35]:
for x in human_to_music_ratio_non_zero_music[:10]:
    pprint(x)

('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/jerry_oconnell__jo_koy_carolla_classics/505.wav',
 0.0,
 0.0,
 47.52983943067222)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/jerry_oconnell__jo_koy_carolla_classics/508.wav',
 0.0,
 0.0,
 170.26732237494645)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/jerry_oconnell__jo_koy_carolla_classics/563.wav',
 0.0,
 0.0,
 71.68524439550173)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/jay_mohr_on_snl_cue_cards_and_ghost_hunters_/262.wav',
 0.0,
 0.0,
 502.2785752293171)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/jillian_michaels_frank_grillo__remy_grillo__news_on/1148.wav',
 0.0,
 0.0,
 142.14853785869275)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/mo_collins__frank_grillo_carolla_classics/2216.wav',
 0.0,
 0.0,
 130.4912221850178)
('/inworld/tts_dataset/podcast_audio_combined/wavs/adam_carolla/wavs/mo

In [36]:
len([x for x in wav_file_and_label_to_energy_level if x[1]['Human sounds'] == 0])

43

In [2]:
from glob import glob
import soundfile as sf

# Get total duration of all files in a folder
ROOT_AUDIO_DIRECTORY = "/inworld/tts_dataset/podcast_audio_combined/wavs"
SPEAKER_NAME = "alec_baldwin"

audio_directory = os.path.join(ROOT_AUDIO_DIRECTORY, SPEAKER_NAME)
wav_files = glob(f"{audio_directory}/**/*.wav", recursive=True)

total_duration = 0
num_errors = 0

print(f"Number of wav files: {len(wav_files)}")

for i, f in enumerate(wav_files):
# for i, f in enumerate(tqdm(wav_files)):
    try:
        duration = sf.info(f).duration
        total_duration += duration
    except:
        num_errors += 1
    
    if i % 10000 == 0:
        print(i)
        print(f"Total duration in folder: {total_duration} seconds, {total_duration / 60 / 60} hours")
        print(f"Number of errors: {num_errors}")


print(f"Total duration in folder: {total_duration} seconds, {total_duration / 60 / 60} hours")
print(f"Number of errors: {num_errors}")

Number of wav files: 101543
Total duration in folder: 2.08 seconds, 0.0005777777777777778 hours
Number of errors: 0
Total duration in folder: 4591.139024943309 seconds, 1.275316395817586 hours
Number of errors: 0
Total duration in folder: 9716.09791383221 seconds, 2.698916087175614 hours
Number of errors: 0
Total duration in folder: 15036.794240362795 seconds, 4.176887288989665 hours
Number of errors: 0
Total duration in folder: 20837.967301587265 seconds, 5.788324250440907 hours
Number of errors: 0
Total duration in folder: 27027.93823129247 seconds, 7.507760619803464 hours
Number of errors: 0
Total duration in folder: 32905.20512471655 seconds, 9.140334756865709 hours
Number of errors: 0
Total duration in folder: 38542.03993197282 seconds, 10.706122203325782 hours
Number of errors: 0
Total duration in folder: 44761.14566893428 seconds, 12.433651574703967 hours
Number of errors: 0
Total duration in folder: 50036.84546485274 seconds, 13.899123740236872 hours
Number of errors: 0
Total d